In [1]:
import numpy as np
import pycameleon

In [2]:
lambdaOrder = [0,7,1,8,2,4,3,9,1,8,0,7,3,6,2,5]
crossTalkCoef = [0.9948,-0.0966,-0.0005,-0.0100,-0.0167,-0.0251,-0.0384,-0.1422,-0.2110,-0.0842,0.0788,1.0723,0.0840,0.0103,0.0147,0.0094,0.0087,-0.0138,-0.0265,0.0141,0.0510,0.1239,0.9772,-0.0789,0.0126,0.0100,-0.0002,-0.0655,-0.0815,0.0115,0.0190,0.0204,0.0352,1.0717,0.1085,0.0058,-0.0100,-0.0465,-0.0488,-0.0481,0.0117,0.0147,0.0211,0.1238,0.9482,-0.0459,-0.0290,-0.0393,-0.0286,-0.0789,-0.0040,-0.0031,-0.0104,-0.0009,0.0279,1.1097,-0.0261,-0.0427,-0.0331,-0.0082,-0.0108,-0.0099,-0.0207,-0.0219,0.0067,0.0175,1.1393,-0.0461,-0.0593,-0.0077,-0.0486,-0.0352,-0.0236,-0.0277,-0.0450,-0.0463,-0.0042,1.4309,-0.0441,-0.1777,-0.0640,-0.0615,-0.0310,-0.0344,-0.0429,-0.0345,-0.0408,0.0162,1.5354,-0.3215,-0.0280,-0.0250,-0.0313,-0.0319,-0.0139,-0.0006,0.0008,-0.0512,-0.0025,1.7008]
crossTalkCoef = np.array(crossTalkCoef).reshape((10,10)) 

In [3]:
# initialize
elementCount = np.zeros(max(lambdaOrder)+1, dtype=int)
for x in range(0,16):
    elementCount[lambdaOrder[x]] += 1

In [4]:
(np.arange(3)+1) @ (np.arange(9)+1).reshape((3,3))

array([30, 36, 42])

In [8]:
elementCount
import os
os.environ['NAPARI_ASYNC'] = '1'

In [9]:
from numba import jit, njit

@njit(nopython=True,parallel=True,fastmath=True,cache=True)
def extractChannel(image,lambdaOrder):

    offSetX = 0
    offSetY = 0
    tVal = np.empty(16, dtype=np.int32)
    #image = np.random.randint(0,255,(2048,2048))
    imgCanal = np.zeros((512,512,max(lambdaOrder)+1),dtype=np.int32)
    for i in range(512):
        for j in range(512):
            tVal[0] = image[i*4+offSetX,j*4+offSetY]
            tVal[1] = image[i*4+offSetX+1,j*4+offSetY]
            tVal[2] = image[i*4+offSetX+2,j*4+offSetY]
            tVal[3] = image[i*4+offSetX+3,j*4+offSetY]
            tVal[4] = image[i*4+offSetX,j*4+offSetY+1]
            tVal[5] = image[i*4+offSetX+1,j*4+offSetY+1]
            tVal[6] = image[i*4+offSetX+2,j*4+offSetY+1]
            tVal[7] = image[i*4+offSetX+3,j*4+offSetY+1]
            tVal[8] = image[i*4+offSetX,j*4+offSetY+2]
            tVal[9] = image[i*4+offSetX+1,j*4+offSetY+2]
            tVal[10] = image[i*4+offSetX+2,j*4+offSetY+2]
            tVal[11] = image[i*4+offSetX+3,j*4+offSetY+2]
            tVal[12] = image[i*4+offSetX,j*4+offSetY+3]
            tVal[13] = image[i*4+offSetX+1,j*4+offSetY+3]
            tVal[14] = image[i*4+offSetX+2,j*4+offSetY+3]
            tVal[15] = image[i*4+offSetX+3,j*4+offSetY+3]
    
            for w in range(len(lambdaOrder)):
                imgCanal[i,j,lambdaOrder[w]] += tVal[w]/elementCount[lambdaOrder[w]]
    
    return imgCanal

In [10]:
@njit(nopython=True,fastmath=True,cache=True)
def applyCrossTalk(img, coef):
    # normalize img
    norm_coef = np.empty(img.shape[2])
    flatten_view = img.reshape((img.shape[0]*img.shape[1],img.shape[2]))
    for i in range(len(norm_coef)):
        norm_coef[i] = 1/flatten_view[:,i].max() 
    img = img*norm_coef.reshape((1,1,-1))
    result = np.empty(img.shape,dtype=np.float32)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]): 
            result[i,j] = img[i,j] @ crossTalkCoef
    return result 
    # return img.max()

In [11]:
@njit(nopython=True,parallel=True,fastmath=True,cache=True)
def calculate_ndvi(img,red_index, nir_index):
    red = img[:,:,red_index].astype(np.float32)
    nir = img[:,:,nir_index].astype(np.float32)
    ndvi = (nir-red)/(nir+red)
    nmin = ndvi.min()
    nmax = ndvi.max()
    # ndvi = (ndvi - nmin)/(nmax-nmin)
    ndvi = (ndvi+np.ones(ndvi.shape))/2
    return np.append(img,ndvi[:,:,np.newaxis],axis=2)
    # return img.max()

In [67]:
# @jit(nopython=True)
#@njit(nopython=True,parallel=True,fastmath=True,cache=True)
def calculate_ndvi2(img,red_index, nir_index):
    epsilon = 1e-6
    red = img[:,:,red_index].astype(np.float32) / 255
    nir = img[:,:,nir_index].astype(np.float32) / 255
    ndvi = np.where((nir + red) == 0, 0, (nir-red)/(nir+red+epsilon))
    ndvi = (ndvi + 1) / 2
    ndvi = (ndvi * 255).astype(np.uint8)
    #ndvi = (ndvi * 255.0).astype(np.int32)
    # print(f"nmin: {red.max()}||\n")
    # print(f"nmax: {nir.max()}//\n")
    #ndvi = (ndvi+np.ones(ndvi.shape))/2
    #nmin = ndvi.min()
    #nmax = ndvi.max()
    # print(img.max())
    
    #ndvi = (ndvi - nmin)/(nmax-nmin)
    # nmin = ndvi.min()
    # nmax = ndvi.max()
    # print(ndvi.dtype)
    # print(f"nmin: {nmin}||\n")
    # print(f"nmax: {nmax}//\n")
    
    return np.append(img,ndvi[:,:,np.newaxis],axis=2)
    # return img.max()

In [13]:
# calculate_ndvi(rr,4,9)[:,:,10].min()

In [14]:
# applyCrossTalk(rr,crossTalkCoef)

In [75]:
import pycameleon

In [76]:
cam = pycameleon.enumerate_cameras()[0]

In [77]:
cam 

CameraInfo { vendor_name: "Imaging Solutions Group", model_name: "CMV4000 Mono", serial_number: "00000000" }

In [18]:
# rr = None
# cam.open()
# cam.load_context()
# payload = cam.start_streaming(3)
# img = cam.receive(payload)
# rr = extractChannel(img,lambdaOrder)
# applyCrossTalk(rr,crossTalkCoef)
# # print(cam.read_enum_as_str(node_name="BalanceWhiteAuto"))
# # print(cam.read_integer(node_name="reg_ExposureAuto"))
# print(cam.read_float(node_name="ExposureTime"))
# print(cam.read_float(node_name="Gain"))
# #updateData(a,rr)
# cam.close()

In [16]:

# import time
# rr = None
# cam.open()
# cam.load_context()
# payload = cam.start_streaming(3)
# for x in range(10000):
#     startTime = time.time()
#     img = cam.receive(payload)
#     recTime = time.time()
#     rr = extractChannel(img,lambdaOrder)
#     endTime = time.time()
#     elapsedRec = recTime - startTime
#     elapsedTime = endTime - startTime
#     print(f'{1/elapsedTime} - {1/elapsedRec} - {1/elapsedTime}',end='\r')
#     time.sleep(0.01)
# cam.close()

In [17]:
# cam.close()

In [18]:
# import cv2

In [19]:
# import napari
# viewer = napari.Viewer()
# a = viewer.add_image(np.zeros((3,3,3)), channel_axis=2)



In [20]:
# a[0].visible = False

In [21]:
# b = viewer.add_image(extractChannel(np.random.randint(0,255,(2048,2048))),channel_axis=2)

In [78]:
def updateData(original, data):
    if len(original) != data.shape[2]:
        raise Exception("Mismatch number of channel")
    for i in range(len(original)):
        original[i].data = data[:,:,i]

from time import sleep
import time
from napari.qt import thread_worker
import os
import sys
import time
from skimage.io.collection import alphanumeric_key
from dask import delayed
import dask.array as da
from tifffile import imread
import napari
from napari.qt import thread_worker


viewer = napari.Viewer(ndisplay=3)

# red 5
# nir 9
band_lut = ['420nm','455nm','518nm','558nm','614nm','659nm','718nm','764nm','821nm','865nm','NDVI']

def append(delayed_image):
    """Appends the image to viewer.

    Parameters
    ----------
    delayed_image : dask.delayed function object
    """
    if delayed_image is None:
        return

    if viewer.layers:
        # layer is present, append to its data
        updateData(viewer.layers,delayed_image)
    else:
        layers = viewer.add_image(delayed_image, channel_axis=2)
        # change color map 
        for i in range(len(layers)):
            l = layers[i]
            l.name = band_lut[i]
            l.visible = False
            l.colormap = "gray"
            #l.contrast_limits=(0,1)

    # we want to show the last file added in the viewer to do so we want to
    # put the slider at the very end. But, sometimes when user is scrolling
    # through the previous slide then it is annoying to jump to last
    # stack as it gets added. To avoid that jump we 1st check where
    # the scroll is and if its not at the last slide then don't move the slider.
    # if viewer.dims.point[0] >= layer.data.shape[0] - 2:
    #     viewer.dims.set_point(0, layer.data.shape[0] - 1)


@thread_worker(connect={'yielded': append})
def watch_path(payload,cam):
    """Watches the path for new files and yields it once file is ready.

    Notes
    -----
    Currently, there is no proper way to know if the file has written 
    entirely. So the workaround is we assume that files are generating 
    serially (in most microscopes it common), and files are name in 
    alphanumeric sequence We start loading the total number of minus the 
    last file (`total__files - last`). In other words, once we see the new 
    file in the directory, it means the file before it has completed so load
    that file. For this example, we also assume that the microscope is 
    generating a `final.log` file at the end of the acquisition, this file 
    is an indicator to stop monitoring the directory.

    Parameters
    ----------
    path : str
    directory to monitor and load tiffs as they start appearing.
    """
    current_files = set()
    processed_files = set()
    end_of_acquisition = False
    while not end_of_acquisition:
        # yield every file to process as a dask.delayed function object.
        startTime = time.time()
        img = cam.receive(payload)
        rr= np.hstack([img[:,:,np.newaxis]])
        rr = extractChannel(img,lambdaOrder)
        rr = applyCrossTalk(rr,crossTalkCoef)
        rr = calculate_ndvi2(rr,5,9)
        endTime = time.time()
        elapsedTime = endTime - startTime
        print(f'{1/elapsedTime} - min{rr[:,:,10].min()} - max{rr[:,:,10].max()}',end='\r')
        sleep(0.02)
        yield rr
    cam.close()

cam.open()
cam.load_context()
print(cam.read_enum_as_str(node_name="BalanceWhiteAuto"))
payload = cam.start_streaming(3)
#cam.write_enum_as_str("PixelFormat","Mono16")
#imm = cam.receive(payload)
print(imm.shape)
print(imm.dtype)
worker = watch_path(payload,cam)
napari.run()

Off
(2048, 2048)
uint8
10.275170383000408 - min9.0 - max249.00

In [79]:
cam.close()

11.485516810795712 - min4.0 - max253.0

Traceback (most recent call last):
  File "/Users/mc/playground/pycameleon/.pixi/envs/default/lib/python3.12/site-packages/superqt/utils/_qthreading.py", line 613, in reraise
    raise e
  File "/Users/mc/playground/pycameleon/.pixi/envs/default/lib/python3.12/site-packages/superqt/utils/_qthreading.py", line 175, in run
    result = self.work()
             ^^^^^^^^^^^
  File "/Users/mc/playground/pycameleon/.pixi/envs/default/lib/python3.12/site-packages/superqt/utils/_qthreading.py", line 440, in work
    output = self._gen.send(_input)
             ^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/y_/djhkgb712wldkrrf149d1b1h0000gn/T/ipykernel_69658/3957524067.py", line 86, in watch_path
    img = cam.receive(payload)
          ^^^^^^^^^^^^^^^^^^^^
ValueError: Failed to receive image


In [44]:
cam.open()
#print(cam.load_context())
print(cam.read_enum_as_str(node_name="PixelFormat"))
cam.write_enum_as_str("PixelFormat","Mono8")
cam.close()

Mono8
